# Portfolio Analysis - Minimum Variance Portfolios

This notebook implements minimum variance portfolios with:
- Three covariance estimation methods (single-factor, constant correlation, shrinkage)
- Portfolios with and without 3% upper bound constraint
- Turnover calculation for all strategies
- Comparison with benchmarks (equal-weight, value-weight)

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

print("Libraries imported successfully!")

## 2. Load Data

In [ ]:
# Load stock returns and market cap data
df = pd.read_csv('/Users/teymour/Desktop/CMOR 462/Final/CMOR462FINAL/Data/monthlyreturncapitalization.csv')
df['Monthly Calendar Date'] = pd.to_datetime(df['Monthly Calendar Date'])
df = df.sort_values(['Ticker', 'Monthly Calendar Date'])
df = df.drop_duplicates(subset=['Ticker', 'Monthly Calendar Date'], keep='last')

returns = df.pivot(index='Monthly Calendar Date', columns='Ticker', values='Monthly Total Return')
mcap = df.pivot(index='Monthly Calendar Date', columns='Ticker', values='Monthly Market Capitalization')

# Load risk-free rate
rf = pd.read_csv('/Users/teymour/Desktop/CMOR 462/Final/CMOR462FINAL/Data/riskfree.csv')
rf['Calendar Date'] = pd.to_datetime(rf['Calendar Date'])
rf = rf.set_index('Calendar Date')['90 Day Bill Returns']

# Load market returns
mkt = pd.read_csv('/Users/teymour/Desktop/CMOR 462/Final/CMOR462FINAL/Data/marketreturn.csv')
mkt['Date'] = pd.to_datetime(mkt['Date'])
mkt = mkt.set_index('Date')['Total Market'].str.rstrip('%').astype(float) / 100

print(f"Returns shape: {returns.shape}")
print(f"Date range: {returns.index[0]} to {returns.index[-1]}")

## 3. Covariance Estimation Functions

In [ ]:
def estimate_covariance_sample(ret_window):
    """Sample covariance estimation."""
    return ret_window.cov()

def estimate_covariance_single_factor(ret_window, mkt_ret):
    """Single-factor (market model) covariance estimation."""
    betas = []
    residual_vars = []
    for col in ret_window.columns:
        y = ret_window[col].dropna()
        x = mkt_ret.loc[y.index]
        valid = ~(y.isna() | x.isna())
        y, x = y[valid], x[valid]
        if len(y) > 1:
            beta = np.cov(x, y)[0,1] / np.var(x)
            residual = y - beta * x
            residual_var = np.var(residual)
        else:
            beta = 0
            residual_var = np.var(y) if len(y) > 0 else 1e-4
        betas.append(beta)
        residual_vars.append(residual_var)
    
    betas = np.array(betas)
    var_mkt = np.var(mkt_ret.loc[ret_window.index].dropna())
    cov_matrix = np.outer(betas, betas) * var_mkt + np.diag(residual_vars)
    return pd.DataFrame(cov_matrix, index=ret_window.columns, columns=ret_window.columns)

def estimate_covariance_constant_corr(ret_window):
    """Constant correlation covariance estimation."""
    corr_matrix = ret_window.corr()
    avg_corr = corr_matrix.where(~np.eye(len(corr_matrix), dtype=bool)).stack().mean()
    std_devs = ret_window.std()
    n = len(std_devs)
    cov_matrix = np.outer(std_devs, std_devs) * avg_corr
    np.fill_diagonal(cov_matrix, std_devs**2)
    return pd.DataFrame(cov_matrix, index=ret_window.columns, columns=ret_window.columns)

def estimate_covariance_shrinkage(ret_window, delta=0.5):
    """Shrinkage covariance estimation."""
    sample_cov = ret_window.cov()
    std_devs = ret_window.std()
    avg_var = np.mean(np.diag(sample_cov))
    target = np.eye(len(std_devs)) * avg_var
    shrunk_cov = delta * target + (1 - delta) * sample_cov.values
    return pd.DataFrame(shrunk_cov, index=ret_window.columns, columns=ret_window.columns)

print("Covariance estimation functions defined!")

## 4. Portfolio Optimization Functions

In [ ]:
def optimize_min_variance(cov_matrix, upper_bound=None):
    """Optimize minimum variance portfolio with optional upper bound constraint."""
    n = len(cov_matrix)
    
    def objective(w):
        return w @ cov_matrix.values @ w
    
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    
    # Add upper bound constraint if specified (e.g., 0.03 for 3% max per stock)
    if upper_bound is not None:
        bounds = tuple((0, upper_bound) for _ in range(n))
    else:
        bounds = tuple((0, 1) for _ in range(n))
    
    w0 = np.ones(n) / n
    
    result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)
    return pd.Series(result.x, index=cov_matrix.columns)

def calculate_turnover(w_old, w_new):
    """Calculate portfolio turnover: sum(|w_new - w_old|) / 2"""
    if w_old is None:
        return 0.0
    
    # Align indices - handle stocks that may enter or exit the universe
    all_stocks = w_old.index.union(w_new.index)
    
    # Create aligned weight vectors (missing stocks get 0 weight)
    w_old_aligned = pd.Series(0.0, index=all_stocks)
    w_new_aligned = pd.Series(0.0, index=all_stocks)
    
    w_old_aligned[w_old.index] = w_old
    w_new_aligned[w_new.index] = w_new
    
    turnover = np.abs(w_new_aligned - w_old_aligned).sum() / 2
    return turnover

print("Optimization functions defined!")

## 5. Backtesting with Upper Bound Constraints and Turnover

In [ ]:
estimation_window = 60
start_date = '2017-01-01'
end_date = '2022-12-31'

oos_dates = returns.loc[start_date:end_date].index

# Results storage - now includes 3% upper bound versions
results = {
    'sample': [],
    'single_factor': [],
    'constant_corr': [],
    'shrinkage': [],
    'single_factor_3pct': [],
    'constant_corr_3pct': [],
    'shrinkage_3pct': [],
    'equal_weight': [],
    'value_weight': []
}

# Track turnover for each strategy
turnovers = {key: [] for key in results.keys()}

# Track previous weights for turnover calculation
prev_weights = {key: None for key in results.keys()}

print("Starting backtesting...")
for i, date in enumerate(oos_dates):
    if i % 12 == 0:
        print(f"Processing {date.strftime('%Y-%m')}...")
    
    date_idx = returns.index.get_loc(date)
    if date_idx < estimation_window:
        continue
    
    # Get estimation window
    ret_window = returns.iloc[date_idx - estimation_window:date_idx]
    ret_window = ret_window.dropna(axis=1, how='all')
    
    # Get out-of-sample returns
    ret_oos = returns.loc[date, ret_window.columns]
    
    # Get market returns for estimation window
    mkt_window = mkt.loc[ret_window.index]
    
    # Estimate covariance matrices
    cov_sample = estimate_covariance_sample(ret_window)
    cov_single = estimate_covariance_single_factor(ret_window, mkt_window)
    cov_const = estimate_covariance_constant_corr(ret_window)
    cov_shrink = estimate_covariance_shrinkage(ret_window)
    
    # Optimize portfolios WITHOUT upper bound
    w_sample = optimize_min_variance(cov_sample)
    w_single = optimize_min_variance(cov_single)
    w_const = optimize_min_variance(cov_const)
    w_shrink = optimize_min_variance(cov_shrink)
    
    # Optimize portfolios WITH 3% upper bound
    w_single_3pct = optimize_min_variance(cov_single, upper_bound=0.03)
    w_const_3pct = optimize_min_variance(cov_const, upper_bound=0.03)
    w_shrink_3pct = optimize_min_variance(cov_shrink, upper_bound=0.03)
    
    # Benchmark portfolios
    w_equal = pd.Series(1.0 / len(ret_window.columns), index=ret_window.columns)
    
    mcap_prev = mcap.iloc[date_idx - 1, :]
    mcap_prev = mcap_prev[ret_window.columns].fillna(0)
    w_value = mcap_prev / mcap_prev.sum()
    
    # Store all current weights
    current_weights = {
        'sample': w_sample,
        'single_factor': w_single,
        'constant_corr': w_const,
        'shrinkage': w_shrink,
        'single_factor_3pct': w_single_3pct,
        'constant_corr_3pct': w_const_3pct,
        'shrinkage_3pct': w_shrink_3pct,
        'equal_weight': w_equal,
        'value_weight': w_value
    }
    
    # Calculate returns and turnovers for all strategies
    for strategy, w in current_weights.items():
        # Portfolio return
        port_return = (w * ret_oos).sum()
        results[strategy].append((date, port_return))
        
        # Turnover
        turnover = calculate_turnover(prev_weights[strategy], w)
        turnovers[strategy].append(turnover)
        
        # Update previous weights
        prev_weights[strategy] = w

print("Backtesting completed!")

## 6. Process Results

In [ ]:
# Convert results to DataFrame
portfolio_returns = {}
for strategy, res in results.items():
    portfolio_returns[strategy] = pd.Series([r[1] for r in res], index=[r[0] for r in res])

returns_df = pd.DataFrame(portfolio_returns)

print(f"Returns dataframe shape: {returns_df.shape}")
print(f"\nStrategies: {list(returns_df.columns)}")
returns_df.head()

## 7. Calculate Performance Metrics (Including Turnover)

In [ ]:
def calculate_metrics(ret_series, rf_series):
    """Calculate performance metrics for a return series."""
    cumulative = (1 + ret_series).cumprod()
    total_return = cumulative.iloc[-1] - 1
    ann_return = (1 + total_return) ** (12 / len(ret_series)) - 1
    ann_vol = ret_series.std() * np.sqrt(12)
    
    rf_aligned = rf_series.loc[ret_series.index]
    excess = ret_series - rf_aligned
    sharpe = excess.mean() / excess.std() * np.sqrt(12)
    
    cummax = cumulative.cummax()
    drawdown = (cumulative - cummax) / cummax
    max_dd = drawdown.min()
    
    return {
        'Total Return': total_return,
        'Ann. Return': ann_return,
        'Ann. Volatility': ann_vol,
        'Sharpe Ratio': sharpe,
        'Max Drawdown': max_dd
    }

# Calculate metrics for all strategies
metrics = {}
for strategy in returns_df.columns:
    metrics[strategy] = calculate_metrics(returns_df[strategy], rf)

metrics_df = pd.DataFrame(metrics).T

# Add average turnover to metrics
avg_turnovers = {strategy: np.mean(turnovers[strategy]) for strategy in turnovers.keys()}
metrics_df['Avg Turnover'] = pd.Series(avg_turnovers)

# Reorder columns for better readability
metrics_df = metrics_df[['Total Return', 'Ann. Return', 'Ann. Volatility', 
                         'Sharpe Ratio', 'Max Drawdown', 'Avg Turnover']]

print("\nPerformance Metrics (with Turnover):")
metrics_df

## 8. Visualizations

In [ ]:
# Cumulative Returns Plot - All 9 Strategies
cumulative_returns = (1 + returns_df).cumprod()

plt.figure(figsize=(16, 8))
for col in cumulative_returns.columns:
    if '3pct' in col:
        # Dashed lines for 3% constrained portfolios
        plt.plot(cumulative_returns.index, cumulative_returns[col], 
                label=col, linewidth=2, linestyle='--')
    else:
        plt.plot(cumulative_returns.index, cumulative_returns[col], 
                label=col, linewidth=2)

plt.legend(loc='best', fontsize=10)
plt.title('Cumulative Returns - All Strategies (Dashed = 3% Upper Bound)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nFinal cumulative returns:")
print(cumulative_returns.iloc[-1].sort_values(ascending=False))

In [ ]:
# Compare portfolios WITH and WITHOUT 3% constraint
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Sharpe Ratio Comparison
sharpe_data = metrics_df['Sharpe Ratio'].sort_values(ascending=False)
colors = ['red' if '3pct' in idx else 'steelblue' for idx in sharpe_data.index]
axes[0].barh(range(len(sharpe_data)), sharpe_data.values, color=colors, alpha=0.7)
axes[0].set_yticks(range(len(sharpe_data)))
axes[0].set_yticklabels(sharpe_data.index, fontsize=9)
axes[0].set_xlabel('Sharpe Ratio', fontsize=11)
axes[0].set_title('Sharpe Ratio Comparison\n(Red = 3% Constraint)', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# Turnover Comparison
turnover_data = metrics_df['Avg Turnover'].sort_values(ascending=False)
colors = ['red' if '3pct' in idx else 'steelblue' for idx in turnover_data.index]
axes[1].barh(range(len(turnover_data)), turnover_data.values, color=colors, alpha=0.7)
axes[1].set_yticks(range(len(turnover_data)))
axes[1].set_yticklabels(turnover_data.index, fontsize=9)
axes[1].set_xlabel('Average Monthly Turnover', fontsize=11)
axes[1].set_title('Turnover Comparison\n(Red = 3% Constraint)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 9. Save Results

In [ ]:
# Save metrics to Excel
metrics_df.to_excel('portfolio_metrics_with_constraints.xlsx')
print("Metrics saved to 'portfolio_metrics_with_constraints.xlsx'")

# Save returns to Excel
returns_df.to_excel('portfolio_returns_with_constraints.xlsx')
print("Returns saved to 'portfolio_returns_with_constraints.xlsx'")